In [ ]:
from neo4j import GraphDatabase

# Replace with your Aura connection details
URI = "neo4j+s://104abb61.databases.neo4j.io"
USER = "neo4j"
PASSWORD = "uSV92xXhnyT0B2nLnSTOaqna4FfQxJHHbdRmqdE___A"

# Connect to the database
driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))

In [10]:
import pandas as pd
import re

# Load the original CSV file
input_file = "fp_growth_association_rules.csv"  # Replace with your file path
output_file = "cleaned_rules.csv"  # Output file name

# Read the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Function to extract movie name and rating from frozenset strings
def extract_movie_and_rating(frozenset_str):
    # Use regex to extract the movie name and rating
    match = re.search(r'''\{(?:'|")(.*)=(.*)(?:'|")\}''', frozenset_str)
    g1 = match.group(1).replace("\"", "")
    g2 = match.group(2).replace("\"", "")
    if match:
        return g1, g2
    return None, None

# Apply the function to the antecedents and consequents columns
df[["antecedent_movie", "antecedent_rating"]] = df["antecedents"].apply(extract_movie_and_rating).apply(pd.Series)
df[["consequent_movie", "consequent_rating"]] = df["consequents"].apply(extract_movie_and_rating).apply(pd.Series)

# Rename columns to remove spaces and make them easier to handle
df.rename(columns={
    "antecedent support": "antecedent_support",
    "consequent support": "consequent_support",
    "zhangs_metric": "zhangs_metric",
    "jaccard": "jaccard",
    "certainty": "certainty",
    "kulczynski": "kulczynski"
}, inplace=True)

# Select relevant columns for the cleaned CSV
cleaned_df = df[[
    "antecedent_movie", "antecedent_rating",
    "consequent_movie", "consequent_rating",
    "support", "confidence", "lift",
    "antecedent_support", "consequent_support",
    "zhangs_metric", "jaccard", "certainty", "kulczynski"
]]

print(cleaned_df['antecedent_movie'][0])
# Save the cleaned data to a new CSV file
cleaned_df.to_csv(output_file, index=False)

print(f"Cleaned data saved to {output_file}")

Forrest Gump (1994)
Cleaned data saved to cleaned_rules.csv


In [ ]:
# Define the Cypher query
query = """
LOAD CSV WITH HEADERS FROM 'https://drive.google.com/uc?export=download&id=1VdBKl-pbt9LSmUKl_D9mu9mMrr_OpeyC' AS row
MERGE (a:Movie {name: row.antecedent_movie, rating: row.antecedent_rating})
MERGE (b:Movie {name: row.consequent_movie, rating: row.consequent_rating})
CREATE (a)-[:BUYS_TOGETHER {
    support: toFloat(row.support),
    confidence: toFloat(row.confidence),
    lift: toFloat(row.lift)
}]->(b)x
"""

# Execute the query
with driver.session() as session:
    session.run(query)
    print("Data loaded successfully!")

Data loaded successfully!


In [14]:
import csv
import ast

# Input and output file paths
input_file = "fp_growth_association_rules.csv"  # Replace with your file path
output_file = "cleaned_rules_grouped.csv"  # Output file name

# Function to extract movie-rating pairs from frozenset strings
def extract_movie_rating_pairs(frozenset_str):
    # Remove 'frozenset' and parse the string as a Python set
    set_str = frozenset_str.replace("frozenset(", "")[:-1]
    movie_rating_set = ast.literal_eval(set_str)
    print(list(movie_rating_set))
    return list(movie_rating_set)

# Open the input and output files
with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8", newline="") as outfile:
    reader = csv.DictReader(infile)
    writer = csv.writer(outfile)

    # Write the header row
    writer.writerow(["group_id", "antecedent_movie", "antecedent_rating", "consequent_movie", "consequent_rating", "support", "confidence", "lift"])

    # Track unique groups
    group_counter = 1

    # Iterate through each row in the input file
    for row in reader:
        # Extract antecedents and consequents
        antecedents = extract_movie_rating_pairs(row["antecedents"])
        consequents = extract_movie_rating_pairs(row["consequents"])

        # Extract metrics
        support = row["support"]
        confidence = row["confidence"]
        lift = row["lift"]

        # Create a unique group ID
        group_id = f"Group{group_counter}"
        group_counter += 1

        # Write rows for the group and its antecedents
        for antecedent in antecedents:
            antecedent_movie, antecedent_rating = antecedent.split("=")
            writer.writerow([group_id, antecedent_movie, antecedent_rating, "", "", "", "", ""])

        # Write rows for the group and its consequents
        for consequent in consequents:
            consequent_movie, consequent_rating = consequent.split("=")
            writer.writerow([group_id, "", "", consequent_movie, consequent_rating, support, confidence, lift])

print(f"Cleaned data saved to {output_file}")

['Forrest Gump (1994)=high']
['Shawshank Redemption, The (1994)=high']
['Shawshank Redemption, The (1994)=high']
['Forrest Gump (1994)=high']
['Pulp Fiction (1994)=high']
['Forrest Gump (1994)=high']
['Forrest Gump (1994)=high']
['Pulp Fiction (1994)=high']
['Silence of the Lambs, The (1991)=high']
['Shawshank Redemption, The (1994)=high']
['Shawshank Redemption, The (1994)=high']
['Silence of the Lambs, The (1991)=high']
['Forrest Gump (1994)=high']
['Silence of the Lambs, The (1991)=high']
['Silence of the Lambs, The (1991)=high']
['Forrest Gump (1994)=high']
['Pulp Fiction (1994)=high']
['Silence of the Lambs, The (1991)=high']
['Silence of the Lambs, The (1991)=high']
['Pulp Fiction (1994)=high']
['Matrix, The (1999)=high']
['Silence of the Lambs, The (1991)=high']
['Silence of the Lambs, The (1991)=high']
['Matrix, The (1999)=high']
['Shawshank Redemption, The (1994)=high']
["Schindler's List (1993)=high"]
["Schindler's List (1993)=high"]
['Shawshank Redemption, The (1994)=high']


In [17]:
query1 = """
// Load antecedent groups and their movies
LOAD CSV WITH HEADERS FROM 'https://drive.google.com/uc?export=download&id=1kQApqFgRmxtbp347wqbHt4bgi9FNvIYA' AS row
WITH row
WHERE row.antecedent_movie IS NOT NULL
MERGE (g:Group {id: row.group_id})
MERGE (a:Movie {name: row.antecedent_movie, rating: row.antecedent_rating})
MERGE (g)-[:CONTAINS]->(a);
"""

query2 = """
// Load consequent movies and their relationships to groups
LOAD CSV WITH HEADERS FROM 'https://drive.google.com/uc?export=download&id=1kQApqFgRmxtbp347wqbHt4bgi9FNvIYA' AS row
WITH row
WHERE row.consequent_movie IS NOT NULL
MERGE (g:Group {id: row.group_id})
MERGE (b:Movie {name: row.consequent_movie, rating: row.consequent_rating})
CREATE (g)-[:LEADS_TO {
    support: toFloat(row.support),
    confidence: toFloat(row.confidence),
    lift: toFloat(row.lift)
}]->(b);
"""

# Execute the queries
with driver.session() as session:
    # Execute the first query
    session.run(query1)
    print("Antecedent groups and movies loaded successfully!")

    # Execute the second query
    session.run(query2)
    print("Consequent movies and relationships loaded successfully!")

Antecedent groups and movies loaded successfully!
Consequent movies and relationships loaded successfully!


In [16]:
from neo4j import GraphDatabase
import pandas as pd

# Replace with your Aura connection details
URI = "neo4j+s://104abb61.databases.neo4j.io"
USER = "neo4j"
PASSWORD = "uSV92xXhnyT0B2nLnSTOaqna4FfQxJHHbdRmqdE___A"
CSV_PATH1 = "Mock_Rule_Set_1.csv"
CSV_PATH2 = "Mock_Rule_Set_2.csv"

# Load CSV
df = pd.read_csv(CSV_PATH1)
df2 = pd.read_csv(CSV_PATH2)


# df['Age_Group'] = df['Age_Group'].replace('-', pd.NA).str.strip()
# df['Gender'] = df['Gender'].replace('-', pd.NA).str.strip()


# Connect to Neo4j
driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))

def create_graph(tx, row):
    # Always create Rule and Diagnosis
    tx.run("""
        MERGE (r:Rule {rule_id: $rule_id})
        MERGE (d:Diagnosis {name: $diagnosis})
        MERGE (r)-[:IMPLIES]->(d)
    """, rule_id=row['Rule_ID'], diagnosis=row['Diagnosis'])

    # Create PatientProfile ONLY IF valid age or gender
    has_valid_age = pd.notna(row['Age_Group']) and row['Age_Group'].strip() not in ["", "-"]
    has_valid_gender = pd.notna(row['Gender']) and row['Gender'].strip() not in ["", "-"]

    if has_valid_age or has_valid_gender:
        props = []
        params = {'rule_id': row['Rule_ID']}
        if has_valid_age:
            props.append('age_group: $age')
            params['age'] = row['Age_Group']
        if has_valid_gender:
            props.append('gender: $gender')
            params['gender'] = row['Gender']

        prop_str = ', '.join(props)

        query = f"""
            MERGE (r:Rule {{rule_id: $rule_id}})
            CREATE (p:PatientProfile {{{prop_str}}})
            MERGE (p)-[:PART_OF]->(r)
        """
        tx.run(query, params)

    # Symptoms
    if pd.notna(row['Symptoms']):
        for symptom in str(row['Symptoms']).split(','):
            symptom = symptom.strip()
            if symptom and symptom not in ["-"]:
                tx.run("""
                    MERGE (s:Symptom {name: $symptom})
                    MERGE (r:Rule {rule_id: $rule_id})
                    MERGE (s)-[:PART_OF]->(r)
                """, symptom=symptom, rule_id=row['Rule_ID'])

    # Lab Tests (new way)
    if pd.notna(row['Lab_Test_Results']):
        for test_result in str(row['Lab_Test_Results']).split(','):
            test_result = test_result.strip()
            if test_result and test_result not in ["-"]:
                # Assuming test_result like: Hemoglobin=Low
                if '=' in test_result:
                    test_name, result_value = test_result.split('=')
                    test_name = test_name.strip()
                    result_value = result_value.strip()
                    tx.run("""
                        MERGE (l:LabTest {name: $test_name})
                        MERGE (r:Rule {rule_id: $rule_id})
                        MERGE (l)-[rel:PART_OF]->(r)
                        SET rel.result = $result_value
                    """, test_name=test_name, result_value=result_value, rule_id=row['Rule_ID'])

def clear_database(tx):
    tx.run("MATCH (n) DETACH DELETE n")


def create_graph_rule2(tx, row):
    # Create Rule and Diagnosis
    tx.run("""
        MERGE (r:Rule {rule_id: $rule_id})
        MERGE (d:Diagnosis {name: $diagnosis})
        MERGE (d)-[:TRIGGERED_BY]->(r)
    """,
    rule_id=row['Rule_ID'],
    diagnosis=row['Diagnosis']
    )

    # Procedures
    if pd.notna(row['Procedures']):
        for proc in str(row['Procedures']).split(','):
            proc = proc.strip()
            if proc and proc not in ["-"]:
                tx.run("""
                    MERGE (p:Procedure {name: $proc})
                    MERGE (r:Rule {rule_id: $rule_id})
                    MERGE (r)-[rel:RECOMMENDS {type: 'Procedure'}]->(p)
                """,
                proc=proc,
                rule_id=row['Rule_ID']
                )

    # Prescriptions
    if pd.notna(row['Prescriptions']):
        for drug in str(row['Prescriptions']).split(','):
            drug = drug.strip()
            if drug and drug not in ["-"]:
                tx.run("""
                    MERGE (m:Prescription {name: $drug})
                    MERGE (r:Rule {rule_id: $rule_id})
                    MERGE (r)-[rel:RECOMMENDS {type: 'Prescription'}]->(m)
                """,
                drug=drug,
                rule_id=row['Rule_ID']
                )


# Run for all rows
with driver.session() as session:
    session.execute_write(clear_database)
    for _, row in df.iterrows():
        session.execute_write(create_graph, row)

    for _, row in df2.iterrows():
        session.execute_write(create_graph_rule2, row)
        
print("✅ Knowledge graph created successfully.")


✅ Knowledge graph created successfully.
